# Accuracy Assessment

To judge how well our classifier does it is critical to provide metrics of how well it does on images it has never seen before. Here we have collected a series of image, "testing images", that were not used in training. 

We organize our testing images in the same file structure as our training dataset:

+ Testing_Images 
    + Buses
        - busimage.jpg
    + Fedex
        - fedeximage.jpg
    + Other
        - otherimage.jpg
        
        
We then need to read in our predicted labels and compare them to actual: 

### Organize Predicted and Actual

In [38]:
import scripts.label_image
import sys
import numpy as np
import tensorflow as tf
import os
from datetime import datetime
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageDraw, ImageFont
import time
import psutil
import pandas as pd
from sklearn.metrics import accuracy_score, cohen_kappa_score


In [28]:

os.chdir(r'/home/mmann1123/Documents/PiCameraClassifier/')
df = pd.read_csv('./predicted_labels.csv',)



In [29]:
# limit to input of interest
df = df[['Path','Date','Class','Prob']]
# add file name column
df['File'] = df['Path'].map(lambda a: os.path.basename(a))
df.head()

,Path,Date,Class,Prob,File
0,/home/mmann1123/Dropbox/Apps/PiCameraLogger/16...,2018-03-16 10:53:00,car,0.911769,Picapture_16_03_2018-10:53:00.jpg
1,/home/mmann1123/Dropbox/Apps/PiCameraLogger/16...,2018-03-16 15:53:00,fedex,0.999535,Picapture_16_03_2018-15:53:00.jpg
2,/home/mmann1123/Dropbox/Apps/PiCameraLogger/16...,2018-03-16 12:38:00,car,0.960822,Picapture_16_03_2018-12:38:00.jpg
3,/home/mmann1123/Dropbox/Apps/PiCameraLogger/16...,2018-03-16 07:23:00,car,0.969589,Picapture_16_03_2018-07:23:00.jpg
4,/home/mmann1123/Dropbox/Apps/PiCameraLogger/16...,2018-03-16 19:15:00,car,0.977345,Picapture_16_03_2018-19:15:00.jpg


In [30]:
# map directory and files into labels
def get_directory_structure(rootdir):
    #Creates a nested dictionary that represents the folder structure of rootdir
    dir = {}
    rootdir = rootdir.rstrip(os.sep)
    start = rootdir.rfind(os.sep) + 1
    for path, dirs, files in os.walk(rootdir):
        folders = path[start:].split(os.sep)
        subdir = dict.fromkeys(files)
        parent = reduce(dict.get, folders[:-1], dir)
        parent[folders[-1]] = subdir
    return dir

dir_dict = get_directory_structure(r'./Testing_Images')

In [31]:
# find class of all Bus testing images
buses = df[df['File'].isin(dir_dict['Testing_Images']['Bus'].keys())] 
# add predicted label 
buses.loc[:,'Pred'] = pd.Series('bus', index=buses.index)
print(buses)

                                                    Path                 Date  \
147    /home/mmann1123/Dropbox/Apps/PiCameraLogger/16...  2018-03-16 09:36:00   
454    /home/mmann1123/Dropbox/Apps/PiCameraLogger/16...  2018-03-16 12:26:00   
483    /home/mmann1123/Dropbox/Apps/PiCameraLogger/16...  2018-03-16 09:45:00   
685    /home/mmann1123/Dropbox/Apps/PiCameraLogger/16...  2018-03-16 09:14:00   
12355  /home/mmann1123/Dropbox/Apps/PiCameraLogger/29...  2018-03-29 09:14:00   
12456  /home/mmann1123/Dropbox/Apps/PiCameraLogger/29...  2018-03-29 07:15:00   
13503  /home/mmann1123/Dropbox/Apps/PiCameraLogger/22...  2018-03-22 11:11:00   
13672  /home/mmann1123/Dropbox/Apps/PiCameraLogger/22...  2018-03-22 16:34:00   
13800  /home/mmann1123/Dropbox/Apps/PiCameraLogger/22...  2018-03-22 16:27:01   
14920  /home/mmann1123/Dropbox/Apps/PiCameraLogger/14...  2018-04-14 08:11:00   
14934  /home/mmann1123/Dropbox/Apps/PiCameraLogger/14...  2018-04-14 08:48:00   
15056  /home/mmann1123/Dropb

In [32]:
# find class of all Fedex testing images
Fedex = df[df['File'].isin(dir_dict['Testing_Images']['Fedex'].keys())] 
# add predicted label 
Fedex.loc[:,'Pred'] = pd.Series('fedex', index=Fedex.index)
#print(Fedex)

In [33]:
# find class of all Fedex testing images
Other = df[df['File'].isin(dir_dict['Testing_Images']['Other'].keys())] 
# add predicted label 
Other.loc[:,'Pred'] = pd.Series('other', index=Other.index)

# rename actual class to other unless it is bus or fedex 
Other.loc[(Other.Class!='fedex') & (Other.Class!='bus'),'Class'] = pd.Series('other', index=Other.index)
#print(Other)

In [34]:
# Create unified df with all predictions and actual
pred_act  = buses.append([Fedex, Other])

### Accuracy Measures

In [35]:
# Confusion Metrics

pd.crosstab(pred_act['Class'], pred_act['Pred'],   colnames=['Predicted']).apply(lambda r: 100.0 * r/r.sum())

Predicted,bus,fedex,other
Class,,,
bus,95.0,0.0,19.047619
car,5.0,5.0,0.000000
fedex,0.0,95.0,19.047619
other,0.0,0.0,61.904762


In [41]:
# Overall accuracy 
accuracy_score(pred_act['Class'], pred_act['Pred'])    

0.83606557377049184

In [39]:
# Kappa 
cohen_kappa_score(pred_act['Class'], pred_act['Pred'])    
    

0.75870253164556956

Looks like there is some confusion between the fedex and bus classes. We can try retraining the model using inception_v3 which should provide more accurate results. Or since both are indications of conjestion, we can leave as is. 